# The fonctions to import


In [19]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette('deep')



# The scraping of the funds of french private company's

In [20]:


def scrape_french_funds():
    url = 'https://www.investing.com/funds/france-funds'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        Name = []
        Symbol = []
        Last = []
        Chg = []
        Total_Assets = []
        Time = []

        table = soup.find('table')
        if table:
            rows = table.find_all('tr')[1:]  
            for row in rows:
                cols = row.find_all('td')
                # print(f"Columns in row: {[col.text.strip() for col in cols]}") # DEBUG PRINT: Show column contents
                if len(cols) >= 8:  
                    Name.append(cols[1].text.strip())  
                    Symbol.append(cols[2].text.strip()) 
                    Last.append(cols[3].text.strip())   
                    Total_Assets.append(cols[5].text.strip()) 
                    Time.append(cols[6].text.strip())   
                    try:
                        Chg_value = float(cols[4].text.strip().replace('%', '')) 
                        Chg.append(Chg_value)
                    except ValueError:
                        Chg.append(None)
                        print(f"Warning: Could not convert '{cols[4].text.strip()}' to float for 'Chg'")

        # Create DataFrame
        df = pd.DataFrame({
            'Name': Name,
            'Symbol': Symbol,
            'Last': Last,
            'Chg': Chg,
            'Total_Assets': Total_Assets,
            'Time': Time
            })

        return df

    except Exception as e:
        print(f'Error scraping data: {e}')
        return None

df = scrape_french_funds()
print(df)

                                         Name   Symbol         Last   Chg  \
0              Amundi Euro Liquidity SRI IC C  0P0000.  249,580.888  0.01   
1              Carmignac Patrimoine A EUR Acc  0P0000.      713.930 -0.03   
2             Carmignac Patrimoine A EUR Ydis  0P0000.      116.710 -0.03   
3              Carmignac Patrimoine E EUR Acc  0P0000.      171.630 -0.03   
4          Carmignac Patrimoine A CHF Acc Hdg  0P0000.      115.010 -0.04   
..                                        ...      ...          ...   ...   
95                               ALM Offensif  0P0000.      310.300 -1.26   
96              29 Haussmann Euro Rendement C  0P0000.    1,607.860  0.10   
97  Candriam Index Arbitrage Classique en EUR  0P0000.    1,520.190  0.12   
98          BNP Paribas Actions Monde Classic  0P0000.       31.120 -0.16   
99   Natixis Ultra Short Term Bonds Plus SI C  0P0000.    1,711.910  0.01   

   Total_Assets   Time  
0        53.84B  13/03  
1         6.07B  13/03  


# Fetch the data


In [21]:
funds_df = scrape_french_funds()

if funds_df is not None:
    print('French Companies Funds Data:')
    display(funds_df)

French Companies Funds Data:


,Name,Symbol,Last,Chg,Total_Assets,Time
0,Amundi Euro Liquidity SRI IC C,0P0000.,"249,580.888",0.01,53.84B,13/03
1,Carmignac Patrimoine A EUR Acc,0P0000.,713.930,-0.03,6.07B,13/03
2,Carmignac Patrimoine A EUR Ydis,0P0000.,116.710,-0.03,6.07B,13/03
3,Carmignac Patrimoine E EUR Acc,0P0000.,171.630,-0.03,6.07B,13/03
4,Carmignac Patrimoine A CHF Acc Hdg,0P0000.,115.010,-0.04,6.07B,13/03
...,...,...,...,...,...,...
95,ALM Offensif,0P0000.,310.300,-1.26,1.12B,11/03
96,29 Haussmann Euro Rendement C,0P0000.,"1,607.860",0.10,1.3B,12/03
97,Candriam Index Arbitrage Classique en EUR,0P0000.,"1,520.190",0.12,256.67M,12/03
98,BNP Paribas Actions Monde Classic,0P0000.,31.120,-0.16,1.36B,12/03


# Risk Analysis

In [22]:
funds_df['Risk'] = funds_df['Chg'].abs()
sorted_df = funds_df.sort_values(by='Risk', ascending=True)

print("Funds from Lowest to Highest Risk:")
display(sorted_df[['Name', 'Risk', 'Total_Assets', 'Chg']])

Funds from Lowest to Highest Risk:


,Name,Risk,Total_Assets,Chg
0,Amundi Euro Liquidity SRI IC C,0.01,53.84B,0.01
5,Carmignac Patrimoine A USD Acc Hdg,0.01,6.07B,-0.01
13,Groupama Entreprises R,0.01,6.25B,0.01
14,Groupama Entreprises N,0.01,6.25B,0.01
12,Groupama Entreprises IC,0.01,6.25B,0.01
...,...,...,...,...
59,Oddo Avenir Europe DR-EUR,1.17,1.25B,1.17
60,Oddo Avenir Europe CI-EUR,1.17,1.25B,1.17
57,Oddo Avenir Europe CN-EUR,1.17,1.25B,1.17
95,ALM Offensif,1.26,1.12B,-1.26


In [31]:
# Top 5 funds with the highest returns
top5_retuns = funds_df.sort_values(by='Chg', ascending=False).head(5)[['Name', 'Chg']]

print("Top 5 Best Performing Funds (by Rate of Change):")
print(top5_getiri.to_string(index=False))

Top 5 Best Performing Funds (by Rate of Change):
                                        Name  Chg
Ofi Financial Investment - Precious Metals R 1.26
                   Oddo Avenir Europe CN-EUR 1.17
                   Oddo Avenir Europe CI-EUR 1.17
                   Oddo Avenir Europe DR-EUR 1.17
                   Oddo Avenir Europe CR-EUR 1.17


In [32]:
# Top 5 funds with the lowest risk
top5_lowest_risk = funds_df.sort_values(by='Risk', ascending=True).head(5)[['Name', 'Risk']]

print("\n Top 5 Least Risky Funds:")
print(top5_lowest_risk.to_string(index=False))


 Top 5 Least Risky Funds:
                              Name  Risk
    Amundi Euro Liquidity SRI IC C  0.01
Carmignac Patrimoine A USD Acc Hdg  0.01
            Groupama Entreprises R  0.01
            Groupama Entreprises N  0.01
           Groupama Entreprises IC  0.01
